<a href="https://colab.research.google.com/github/KTH-EXPECA/examples/blob/main/wireless-pr3d/ep5g_latency_measurement_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Login to Chameleon and download openrc.sh file from [here](https://testbed.expeca.proj.kth.se/project/api_access/openrc/). Upload it here next to this notebook and continue.

In the next cell, we setup the authentication method to be able to use Openstack clients.

In [1]:
import os, re
from getpass import getpass

with open('/content/sdr-test-project-openrc.sh', 'r') as f:
    script_content = f.read()
    pattern = r'export\s+(\w+)\s*=\s*("[^"]+"|[^"\n]+)'
    matches = re.findall(pattern, script_content)

    for name, value in matches:
        os.environ[name] = value.strip('"')

password = getpass('enter your expeca password:')
os.environ['OS_PASSWORD'] = password

enter your expeca password:··········


Install required packages and dependencies. Ignore the warnings.

In [2]:
!pip uninstall -q -y moviepy
!pip install -q jedi
!pip install -q git+https://github.com/KTH-EXPECA/python-chi

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done


Import packages

In [3]:
import json
from loguru import logger
import chi.network, chi.container, chi.network
from chi.expeca import reserve, list_reservations, unreserve_byid, get_container_status, wait_until_container_removed

In the next cell, we reserve the required equipment and resources to form an end to end experiment setup. We reserve EP5G network, one Advantech router and one worker to run the workloads.

In [4]:
# ep5g reservation
ep5g_lease = reserve(
    { "type":"network", "name": "ep5g", "net_name": "ep5g-vip", "segment_id": "300", "duration": { "days":7, "hours":0 } }
)

# advantech router reservation
adv_lease = reserve(
    { "type":"network", "name": "adv-02", "net_name": "adv-02", "segment_id": "131", "duration": { "days":7, "hours":0 } }
)

# worker reservation
worker_lease = reserve(
    { "type":"device", "name":"worker-01", "duration": { "days":7, "hours":0 } }
)

worker_reservation_id = worker_lease["reservations"][0]["id"]

leaseslist = list_reservations(brief=True)
print(json.dumps(leaseslist,indent=4))

2023-07-31 21:30:27.948 | INFO     | chi.expeca:reserve:156 - reserving ep5g
2023-07-31 21:30:32.445 | INFO     | chi.expeca:wait_until_lease_status:62 - waiting 120 seconds for ep5g-lease with id d4b3d6cb-ce41-476d-bccd-f32accfc04d1 to become "ACTIVE"
2023-07-31 21:30:37.608 | INFO     | chi.expeca:wait_until_lease_status:69 - lease ep5g-lease with id d4b3d6cb-ce41-476d-bccd-f32accfc04d1 is PENDING.
2023-07-31 21:30:42.784 | INFO     | chi.expeca:wait_until_lease_status:69 - lease ep5g-lease with id d4b3d6cb-ce41-476d-bccd-f32accfc04d1 is PENDING.
2023-07-31 21:30:47.958 | INFO     | chi.expeca:wait_until_lease_status:69 - lease ep5g-lease with id d4b3d6cb-ce41-476d-bccd-f32accfc04d1 is PENDING.
2023-07-31 21:30:53.133 | INFO     | chi.expeca:wait_until_lease_status:69 - lease ep5g-lease with id d4b3d6cb-ce41-476d-bccd-f32accfc04d1 is PENDING.
2023-07-31 21:30:58.297 | INFO     | chi.expeca:wait_until_lease_status:69 - lease ep5g-lease with id d4b3d6cb-ce41-476d-bccd-f32accfc04d1 is P

[
    {
        "name": "adv-02-lease",
        "id": "4a0813f3-6da9-450a-94ed-8d59ae0cfe41",
        "reservation_id": "2396ddca-b523-4faf-8448-7d020855ebb2",
        "status": "ACTIVE",
        "end_date": "2023-08-07T21:31:00.000000"
    },
    {
        "name": "ep5g-lease",
        "id": "d4b3d6cb-ce41-476d-bccd-f32accfc04d1",
        "reservation_id": "e7121942-4641-4747-be7f-10b06efc9fe7",
        "status": "ACTIVE",
        "end_date": "2023-08-07T21:30:00.000000"
    },
    {
        "name": "worker-01-lease",
        "id": "ec796b79-7f39-4890-b477-69f704c515e8",
        "reservation_id": "374f891b-9ac9-4605-8c6d-00c2b4404ad2",
        "status": "ACTIVE",
        "end_date": "2023-08-07T21:32:00.000000"
    }
]


In the following section we setup the networking equipment for ep5g (refer to [here](https://kth-expeca.gitbook.io/testbedconfig/enroll/enroll-network-segments/ep5g) for more info).
It contains creation of an edge-net, a router, and some interfaces on the router and routes.

In [5]:
# create edge-net
edgenet = chi.network.create_network("edge-net")
chi.network.create_subnet("edge-net-subnet",edgenet["id"],"10.70.70.0/24",gateway_ip="10.70.70.1",enable_dhcp=False)
logger.success("edge-net is created.")

# create ep5g-vip-router
router = chi.network.create_router("ep5g-vip-router","public")
logger.success("ep5g-vip-router router is created.")
logger.info(f"{json.dumps(router,indent=4)}")

# connect ep5g-vip-net to ep5g-vip-router
ep5gnet = chi.network.get_network("ep5g-vip-net")
portadd = chi.network.add_subnet_to_router(router["id"],ep5gnet["subnets"][0])
logger.success("An interface on ep5g-vip-net is added to the router")

# create edge-net to ep5g-vip-router
edgenet = chi.network.get_network("edge-net")
portadd = chi.network.add_subnet_to_router(router["id"],edgenet["subnets"][0])
logger.success("An interface on edge-net is added to the router")

# add ep5g route to ep5g-vip-router
routeadd = chi.network.add_route_to_router(router["id"],"172.16.0.0/16","10.30.111.10")
logger.success("the route added to the router")

2023-07-31 21:33:25.312 | SUCCESS  | __main__:<cell line: 4>:4 - edge-net is created.
2023-07-31 21:33:28.054 | SUCCESS  | __main__:<cell line: 8>:8 - ep5g-vip-router router is created.
2023-07-31 21:33:28.057 | INFO     | __main__:<cell line: 9>:9 - {
    "id": "29496bed-14d5-48d4-98a0-598ec2bdbfad",
    "name": "ep5g-vip-router",
    "tenant_id": "f2831b1828814f5db878f45a61d1043c",
    "admin_state_up": true,
    "status": "ACTIVE",
    "external_gateway_info": {
        "network_id": "717b5f2b-069e-4868-a24d-91a4ae3ad002",
        "external_fixed_ips": [
            {
                "subnet_id": "53d03ffd-0d49-4f9a-88a3-a30d69fe4827",
                "ip_address": "130.237.11.113"
            }
        ],
        "enable_snat": true
    },
    "description": "",
    "availability_zones": [],
    "availability_zone_hints": [],
    "routes": [],
    "flavor_id": null,
    "tags": [],
    "created_at": "2023-07-31T21:33:25Z",
    "updated_at": "2023-07-31T21:33:26Z",
    "revision_num

Now the network is ready to run the workloads. We start by runming the edge-node perf-meas container.

In [6]:
edgenet = chi.network.get_network("edge-net")
chi.container.create_container(
    name = "edge-node",
    image = "samiemostafavi/perf-meas",
    reservation_id = worker_reservation_id,
    environment = {"SERVER_DIR":"/mnt/volume/"},
    mounts = [
        {'source': 'edge-volume', 'destination': '/mnt/volume/'}
    ],
    nets = [
        { "network" : edgenet['id'] },
    ],
    labels = {
        "networks.1.interface":"eno12419",
        "networks.1.ip":"10.70.70.3/24",
        "networks.1.routes":"172.16.0.0/16-10.70.70.1",
    },
)
chi.container.wait_for_active("edge-node")
logger.success("created edge-node container.")

2023-07-31 21:34:11.960 | SUCCESS  | __main__:<cell line: 20>:20 - created edge-node container.


Next, we run the end-node perf-meas container.

In [7]:
advnet = chi.network.get_network("adv-02-net")
chi.container.create_container(
    name = "end-node",
    image = "samiemostafavi/perf-meas",
    reservation_id = worker_reservation_id,
    environment = {"SERVER_DIR":"/tmp/"},
    nets = [
        { "network" : advnet['id'] },
    ],
    labels = {
        "networks.1.interface":"eno12429",
        "networks.1.ip":"10.42.3.2/24",
        "networks.1.routes":"10.70.70.0/24-10.42.3.1",
    },
)
chi.container.wait_for_active("end-node")
logger.success("created end-node container.")

2023-07-31 21:34:37.099 | SUCCESS  | __main__:<cell line: 17>:17 - created end-node container.


Run uplink bandwidth test

In [ ]:
command = "iperf3 -c 10.70.70.3 -u -b 1G --get-server-output > /proc/1/fd/1 2>&1"
result = chi.container.execute(
    container_ref="end-node",
    command="curl -s -X POST -H \"Content-Type: application/json\" -d '{\"cmd\": \"" + command + "\"}' http://localhost:50505/",
)
logger.info(f"{result}")

Run downlink bandwidth test

In [ ]:
command = "iperf3 -c 172.16.0.88 -u -b 1G --get-server-output > /proc/1/fd/1 2>&1"
result = chi.container.execute(
    container_ref="edge-node",
    command="curl -s -X POST -H \"Content-Type: application/json\" -d '{\"cmd\": \"" + command + "\"}' http://localhost:50505/",
)
logger.info(f"{result}")

CAUTION: In this cell we tear down all the configurations and release the reserved resources. The project will be clean afterwards.

In [8]:

status = get_container_status("edge-node")
if status:
    chi.container.destroy_container("edge-node")
    wait_until_container_removed("edge-node")


status = get_container_status("end-node")
if status:
    chi.container.destroy_container("end-node")
    wait_until_container_removed("end-node")

logger.info(f"stopped and removed all containers")

# find the router again
router = None
try:
    router = chi.network.get_router("ep5g-vip-router")
except Exception as ex:
    logger.info(f"could not find ep5g-vip-router.")

if router:
    # remove all routes from the router
    chi.network.remove_all_routes_from_router(router["id"])
    logger.success(f"removed all routers from router")

    # remove all subnets from the router
    subnets = chi.network.list_subnets()
    logger.info(f"checking all {len(subnets)} subnets.")
    for subnet in subnets:
        try:
            chi.network.remove_subnet_from_router(router["id"],subnet["id"])
        except Exception as ex:
            pass
    logger.success(f"removed all subnets from router")

    chi.network.delete_router(router["id"])
    logger.success(f"deleted the router")

edgenet = None
try:
    edgenet = chi.network.get_network("edge-net")
except Exception as ex:
    logger.info(f"could not find edge-net.")

if edgenet:
    chi.network.delete_network(edgenet["id"])
    logger.success(f"deleted the edge-net")

# remove the leases
unreserve_byid(ep5g_lease["id"])
unreserve_byid(adv_lease["id"])
unreserve_byid(worker_lease["id"])

2023-07-31 21:34:48.557 | INFO     | chi.expeca:wait_until_container_removed:30 - waiting 30 seconds for edge-node container to be removed
2023-07-31 21:34:53.745 | INFO     | chi.expeca:wait_until_container_removed:37 - container edge-node is in Deleting state.
2023-07-31 21:34:58.944 | INFO     | chi.expeca:wait_until_container_removed:37 - container edge-node is in None state.
2023-07-31 21:34:59.392 | INFO     | chi.expeca:wait_until_container_removed:30 - waiting 30 seconds for end-node container to be removed
2023-07-31 21:35:04.588 | INFO     | chi.expeca:wait_until_container_removed:37 - container end-node is in Deleting state.
2023-07-31 21:35:09.775 | INFO     | chi.expeca:wait_until_container_removed:37 - container end-node is in None state.
2023-07-31 21:35:09.778 | INFO     | __main__:<cell line: 12>:12 - stopped and removed all containers
2023-07-31 21:35:12.739 | SUCCESS  | __main__:<cell line: 21>:24 - removed all routers from router
2023-07-31 21:35:12.950 | INFO     |